In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

from matplotlib import gridspec
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Layer, concatenate
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.layers import BatchNormalization


plt.rc('font', size=20)
plt.rcParams["font.family"] = "serif"

In [ ]:
normalize = True
data_size = 10**6
n_moments = 2
mylambda = []

In [ ]:
moment = 2
obs = 'q'

In [ ]:
#load and normalize the data
data = np.load('npfiles/rawdata.npz')
substructure_variables = ['pT', 'w', 'q', 'm', 'r', 'tau1s', 'tau2s']
data_streams = ['_true', '_true_alt', '_reco', '_reco_alt']
n_variables = len(substructure_variables)


normalize = True
    
for stream in data_streams:
    globals()['pT'+stream] = data['pT'+stream][:150000]
    globals()['x'+stream] = data[obs+stream][:150000]
    
xm, xs = (x_true_alt.mean(), x_true_alt.std()) if normalize else (0, 1)
pm, ps = (pT_true_alt.mean(), pT_true_alt.std()) if normalize else (0, 1)

for stream in data_streams:
    globals()['x'+stream] = (globals()['x'+stream] - xm)/xs
    globals()['pT'+stream] = (globals()['pT'+stream] - pm)/ps

In [ ]:
class MyLayer(Layer):

    def __init__(self, myc, **kwargs):
        self.myinit = myc
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.m0 = self.add_weight(name='m0', 
                                    shape=(1,),
                                    initializer=tf.keras.initializers.Constant(self.myinit[0]), 
                                    trainable=True)
        self.m1 = self.add_weight(name='m1', 
                                    shape=(1,),
                                    initializer=tf.keras.initializers.Constant(self.myinit[1]), 
                                    trainable=True)
        self.v0 = self.add_weight(name='v0', 
                                    shape=(1,),
                                    initializer=tf.keras.initializers.Constant(self.myinit[2]), 
                                    trainable=True)
        self.v1 = self.add_weight(name='v1', 
                                    shape=(1,),
                                    initializer=tf.keras.initializers.Constant(self.myinit[3]), 
                                    trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        y = tf.exp((self.m0 + self.m1*x[:, 0]) * x[:,1] + (self.v0 + self.v1*x[:,0]) * x[:,1]**2)
        return tf.reshape(y, (len(x), 1))

In [ ]:
def weighted_mlc(y_true, y_pred):
    weights = tf.gather(y_true, [1], axis=1) # event weights
    y_true = tf.gather(y_true, [0], axis=1) # actual y_true for loss
    
    weights_1 = K.sum(y_true*weights)
    weights_0 = K.sum((1-y_true)*weights)
    
    # Clip the prediction value to prevent NaN's and Inf's
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    t_loss = -weights * ((y_true) * K.log(y_pred)/weights_1 +
                         (1 - y_true) * (1 - y_pred)/weights_0)
    return K.mean(t_loss)

def weighted_mlc_GAN(y_true, y_pred):
    weights = tf.gather(y_pred, [1], axis=1) # event weights
    y_pred = tf.gather(y_pred, [0], axis=1) # actual y_pred for loss
    
    weights_1 = K.sum(y_true*weights)
    weights_0 = K.sum((1-y_true)*weights)
    
    
    # Clip the prediction value to prevent NaN's and Inf's
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    t_loss = weights * ((1 - y_true) * (1 - y_pred)/weights_0)
    return K.mean(t_loss)

In [ ]:
myc = np.random.normal(0, 0.01, 4)
mymodel_inputtest = Input(shape=(2,))
mymodel_test = MyLayer(myc)(mymodel_inputtest)
model_generator = Model(mymodel_inputtest, mymodel_test)

inputs_disc = Input((2, ))
hidden_layer_1_disc = Dense(50, activation='relu')(inputs_disc)
hidden_layer_2_disc = Dense(50, activation='relu')(hidden_layer_1_disc)
hidden_layer_3_disc = Dense(50, activation='relu')(hidden_layer_2_disc)
outputs_disc = Dense(1, activation='sigmoid')(hidden_layer_3_disc)
model_discrimintor = Model(inputs=inputs_disc, outputs=outputs_disc)

model_discrimintor.compile(loss=weighted_mlc, optimizer=tf.keras.optimizers.Adam())
    
model_discrimintor.trainable = False
mymodel_gan = Input(shape=(2,))
gan_model = Model(inputs=mymodel_gan,outputs=concatenate([model_discrimintor(mymodel_gan),model_generator(mymodel_gan)]))

gan_model.compile(loss=weighted_mlc_GAN, optimizer=tf.keras.optimizers.Adam())

In [ ]:
xvals_particle = np.transpose([np.concatenate([x_true_alt,x_true]), np.concatenate([pT_true_alt,pT_true])])
xvals_detector = np.transpose([np.concatenate([x_reco_alt,x_reco]), np.concatenate([pT_reco_alt,pT_reco])])                        
yvals = np.transpose(np.concatenate([np.ones(len(x_true_alt)),np.zeros(len(x_true))]))

X_train_particle, X_test_particle, X_train_detector, X_test_detector, Y_train, Y_test = train_test_split(xvals_particle, 
                                                                                                        xvals_detector,
                                                                                                        yvals)

betas = []

In [ ]:
n_epochs = 50
n_batch = 128*100
n_batches = len(X_train_particle) // n_batch

for i in range(n_epochs):
    #print("  ",np.sum(model_generator.predict(X_train_1,batch_size=1000)))
    for j in range(n_batches):
        X_batch_particle = X_train_particle[j*n_batch:(j+1)*n_batch]
        X_batch_detector = X_train_detector[j*n_batch:(j+1)*n_batch]
        Y_batch = Y_train[j*n_batch:(j+1)*n_batch]
        W_batch = model_generator(X_batch_particle)
        W_batch = np.array(W_batch).flatten()
        
        W_batch[Y_batch==1] = 1        
        Y_batch_2 = np.stack((Y_batch, W_batch), axis=1)
        
        model_discrimintor.train_on_batch(X_batch_detector, Y_batch_2)        
        gan_model.train_on_batch(X_batch_particle[Y_batch==0],Y_batch[Y_batch==0])
    betas += [np.array(model_generator.layers[-1].get_weights())]
    if i%20 == 0:
        print("on epoch=", i, np.mean(betas))
    if np.isnan(np.mean(betas)):
        break

In [ ]:
truth = X_test_particle[Y_test==0]
gen = X_test_particle[Y_test==1]
data = X_test_detector[Y_test==0]
sim = X_test_detector[Y_test==1]

total_err = []

for mylambda in betas:
    arr = np.exp(
        (mylambda[0] + mylambda[1] * gen[:, 1]) * gen[:, 0]
        + (mylambda[2] + mylambda[3]*gen[:, 1]) * gen[:, 0]**2
    )
    weights_moment_uf = (arr*len(data)/np.sum(arr))

    mean_err = (np.average(gen[:, 0], weights = weights_moment_uf) - truth[:, 0].mean())
    var_err = np.average(gen[:, 0]**2, weights = weights_moment_uf) - np.mean(truth[:, 0]**2)
    total_err += [(mean_err)**2 + var_err]
    
mylambda = betas[np.argmin(total_err)]
arr = np.exp(
    (mylambda[0] + mylambda[1] * gen[:, 1]) * gen[:, 0]
    + (mylambda[2] + mylambda[3]*gen[:, 1]) * gen[:, 0]**2
    )
weights_moment_uf = arr*len(data)/np.sum(arr)

In [ ]:
truth = X_test_particle[Y_test==0][:, 0] * xs + xm
gen = X_test_particle[Y_test==1][:, 0]* xs + xm
data = X_test_detector[Y_test==0][:, 0]* xs + xm
sim = X_test_detector[Y_test==1][:, 0]* xs + xm


bins = np.linspace(truth.min(), truth.max(), 30)

fig, ax = plt.subplots(figsize=(14, 7))

_,_,_=plt.hist(truth, bins=bins, alpha=0.5, label="Truth", density=True)
_,_,_=plt.hist(gen, bins=bins, alpha=0.5, label="Generation", density=True)
_,_,_=plt.hist(gen, bins=bins, weights=weights_moment_uf, histtype="step", color='black', ls=":", lw=4, label="Moment Unfolding", density=True)

plt.legend(fontsize=24)
plt.xlabel("z (particle level)", fontsize=24)
plt.ylabel("Counts", fontsize=24)
plt.title(f"Jet {obs} Data: $p_T$ Conditioned Inclusive Histograms", fontsize=24)

plt.savefig(f"figures/{obs}pjetexample.pdf", bbox_inches='tight', transparent=True)
plt.show()

## Set up binning

In [ ]:
binvals = [pT_true.min()]
i = 0
while binvals[-1] < pT_true.max() and i < len(binvals):
    for binhigh in np.linspace(binvals[i] + 0.01, pT_true.max(), 100):
        in_bin = (pT_true > binvals[i]) & (pT_true < binhigh)
        in_reco_bin = (pT_reco > binvals[i]) & (pT_reco < binhigh)
        if np.sum(in_bin) > 0:
            purity = np.sum(in_bin & in_reco_bin) / np.sum(in_bin)
            if purity > np.sqrt(0.5):
                print(f"{binhigh = }, {purity = }")
                i += 1
                binvals.append(binhigh)
                break
    else:
        break

print(f"{len(binvals) = }")

In [ ]:
n_bins =len(binvals) - 1

In [ ]:
pTbin_truth = np.clip(np.digitize(pT_true,binvals),1,n_bins)-1
pTbin_reco = np.clip(np.digitize(pT_reco,binvals),1,n_bins)-1
pTbin_truth_alt = np.clip(np.digitize(pT_true_alt,binvals),1, n_bins)-1
pTbin_reco_alt = np.clip(np.digitize(pT_reco_alt,binvals),1, n_bins)-1

In [ ]:
binvals = np.array(binvals)
binmid = (binvals[:-1] + binvals[1:])/2

In [ ]:
pTbin_gen_test = (np.clip(np.digitize(X_test_particle[:,1],binvals),1,n_bins)-1)[Y_test==0]
pTbin_truth_test = (np.clip(np.digitize(X_test_particle[:,1],binvals),1,n_bins)-1)[Y_test==1]

In [ ]:
means_moment_unfolding = np.empty(n_bins)
truth_p = np.empty(n_bins)
gen_p = np.empty(n_bins)

truth_std = np.array([np.std(truth**moment) for i in range(n_bins)])
n_array = np.array([len(truth[pTbin_truth == i]) for i in range(n_bins)])

for i in range(n_bins):
    means_moment_unfolding[i] = np.average((gen[pTbin_gen_test==i])**moment, weights = weights_moment_uf)
    gen_p[i] = np.average((gen[pTbin_gen_test==i])**moment)
    truth_p[i] = np.average((truth[pTbin_truth_test==i])**moment)

In [ ]:
drop = 2
markersize = 10
def plot_data(ax, y, label, marker, color):
    x = binmid[:-drop]*ps + pm
    y = y[:-drop]*xs + xm
    
    if label == 'Truth':
        yerr = (truth_std/np.sqrt(n_array))[:-drop]
        ax.errorbar(x, y, yerr=yerr, linestyle='--', color=color, alpha=1, capsize=3, 
                       label=label, marker=marker, mec=color, mfc='none', ms=markersize)
        ax.plot(x, y, linestyle='--', 
                   color=color, alpha = 1, marker=marker,
                  mec=color, mfc='none', ms=markersize)
    else:    
        ax.plot(x, y, linestyle='--', 
                   color=color, alpha = 1, label=label, marker=marker,
                  mec=color, mfc='none', ms=markersize)

In [ ]:
datasets = [
    (means_moment_unfolding, 'Moment Unfolding', 'o', 'black'),
    (truth_p, 'Truth', 'v', 'blue'),
    (gen_p, 'Generation', '^', 'orange')
]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# Plot settings for ax0
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')
ax.tick_params(direction="in", which="both")
ax.minorticks_on()
#ax[0].set_ylim(0.0, 0.2)
#ax.set_xlim(100.0, 700)

for sets, label, marker, color in datasets:
    plot_data(ax, sets, label, marker, color)

ylabel = r'$\langle {} \rangle$'.format(obs.upper()) if moment == 1 else r'$\langle {}^{} \rangle$'.format(obs.upper(), moment)
if obs == 'm':
    unit = ' [GeV]' if moment == 1 else ' [GeV$^{}$]'.format(moment)
    ylabel += unit
ax.set_ylabel(ylabel, fontsize=28)

ax.legend(frameon=True, fontsize=28)
ax.set_xlabel("Jet $p_{T}$ [GeV]", fontsize=28)



fig.savefig('figures/p{}jet{}.pdf'.format(obs, moment), bbox_inches='tight')
plt.show()